In [25]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [26]:
dataset = pd.read_csv('Churn_Modelling.csv', index_col = 'RowNumber')


In [27]:
X_columns = dataset.columns.tolist()[2:12]
Y_columns = dataset.columns.tolist()[-1:]
print(X_columns)
print(Y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [28]:
X = dataset[X_columns].values
Y = dataset[Y_columns].values

In [29]:
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])

In [30]:
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

In [31]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ],
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [32]:
X = pipeline.fit_transform(X)

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

2023-10-20 03:31:48.383079: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-20 03:31:48.411875: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-20 03:31:48.411907: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-20 03:31:48.411942: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-20 03:31:48.418801: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-20 03:31:48.431424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [35]:
classifier = Sequential()

In [36]:
classifier.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
classifier.add(Dropout(rate = 0.1))

In [37]:
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [38]:
classifier.add(Dense(1, activation = 'sigmoid'))

In [39]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [40]:
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split = 0.1, verbose = 2)

Epoch 1/200


225/225 - 1s - loss: 0.5291 - accuracy: 0.7962 - val_loss: 0.4869 - val_accuracy: 0.7950 - 1s/epoch - 5ms/step
Epoch 2/200
225/225 - 0s - loss: 0.4922 - accuracy: 0.7954 - val_loss: 0.4630 - val_accuracy: 0.7950 - 270ms/epoch - 1ms/step
Epoch 3/200
225/225 - 0s - loss: 0.4728 - accuracy: 0.7960 - val_loss: 0.4494 - val_accuracy: 0.7950 - 260ms/epoch - 1ms/step
Epoch 4/200
225/225 - 0s - loss: 0.4597 - accuracy: 0.7969 - val_loss: 0.4402 - val_accuracy: 0.7937 - 289ms/epoch - 1ms/step
Epoch 5/200
225/225 - 0s - loss: 0.4538 - accuracy: 0.7976 - val_loss: 0.4330 - val_accuracy: 0.7987 - 333ms/epoch - 1ms/step
Epoch 6/200
225/225 - 0s - loss: 0.4449 - accuracy: 0.8026 - val_loss: 0.4258 - val_accuracy: 0.8025 - 281ms/epoch - 1ms/step
Epoch 7/200
225/225 - 0s - loss: 0.4358 - accuracy: 0.8053 - val_loss: 0.4166 - val_accuracy: 0.8050 - 243ms/epoch - 1ms/step
Epoch 8/200
225/225 - 0s - loss: 0.4289 - accuracy: 0.8097 - val_loss: 0.4082 - val_accuracy: 0.8087 - 254ms/epoch - 1ms/step
Epoch 9

In [41]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 [==============================] - 0s 764us/step


[[0.24358848]
 [0.3375278 ]
 [0.18739332]
 [0.0810818 ]
 [0.11788049]]


In [42]:
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [43]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1516   79]
 [ 195  210]]


In [44]:
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

86.3 % of data was classified correctly
